In [1]:
from azureml.core import Workspace

In [2]:
ws = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F6XAZTSEA to authenticate.
Interactive authentication successfully completed.


In [3]:
ws.name

'aml-ws'

In [4]:
ws.datasets

{}

In [5]:
ws.datastores

{'workspaceblobstore': <azureml.data.azure_storage_datastore.AzureBlobDatastore at 0x7f52a29be940>,
 'workspacefilestore': <azureml.data.azure_storage_datastore.AzureFileDatastore at 0x7f52a294d4e0>}

In [6]:
ws.get_default_datastore()

In [7]:
for ds_name in ws.datastores:
    print(ds_name)

workspaceblobstore
workspacefilestore


In [8]:
import os

experiment_folder = "training_folder"
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, "folder created")

training_folder folder created


In [20]:
%%writefile $experiment_folder/train.py

import pandas as pd
import numpy as np

from azureml.core import Run
#import joblib

import keras
from tensorflow.keras.models import Sequential 

from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam, RMSprop
from keras import backend as k
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist 

run = Run.get_context()

(x_train, y_train) , (x_test, y_test) = mnist.load_data()
num_labels = len(np.unique(y_train))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


image_size = x_train.shape[1]
input_size = image_size * image_size

x_train = np.reshape(x_train, [-1, input_size])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, input_size])
x_test = x_test.astype('float32') / 255

batch_size = 128
hidden_units = 256
dropout = 0.45

model = Sequential()
model.add(Dense(hidden_units, input_dim=input_size))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(hidden_units))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])


model.fit(x_train, y_train, epochs=10, batch_size=batch_size)

train_loss, train_acc = model.evaluate(x_train, y_train, batch_size=batch_size)
print('Train Accuracy:', train_acc)
run.log('Train Accuracy', np.float(train_acc))

loss, test_acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test Accuracy:', test_acc)
run.log('Test Accuracy', np.float(test_acc))

model.save("outputs/my_h5_model.h5")
run.complete()

Overwriting training_folder/train.py


In [21]:
compute_target = ws.compute_targets['aml-compute']

In [22]:
#create an estimator
from azureml.train.dnn import TensorFlow

est = TensorFlow(source_directory=experiment_folder,
                entry_script='train.py',
                compute_target=compute_target,
                pip_packages=['tensorflow==2.2', 'keras' ])

WARNING - framework_version is not specified, defaulting to version 1.13.
WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['tensorflow']. We cannot guarantee image build will succeed.


In [23]:
from azureml.core import Experiment
exp = Experiment(workspace=ws, name='keras-mnist')

In [24]:
run = exp.submit(est)
run.wait_for_completion(show_output=True)

RunId: keras-mnist_1594201764_d82072c2
Web View: https://ml.azure.com/experiments/keras-mnist/runs/keras-mnist_1594201764_d82072c2?wsid=/subscriptions/23b3eeaf-8619-495c-b837-5dece9a29613/resourcegroups/aml-rg1/workspaces/aml-ws

Streaming azureml-logs/55_azureml-execution-tvmps_f123fe9481f13ff58d15731b92e96095afc81d3d00e719a753089e9619330ad8_p.txt

2020-07-08T09:53:45Z Executing 'Copy ACR Details file' on 10.0.0.5
2020-07-08T09:53:45Z Copy ACR Details file succeeded on 10.0.0.5. Output: 
>>>   
>>>   
2020-07-08T09:53:45Z Starting output-watcher...
2020-07-08T09:53:45Z IsDedicatedCompute == False, starting polling for Low-Pri Preemption
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_66dacb1972b3b6e7c0f4d448c6ae29b4
a1298f4ce990: Pulling fs layer
04a3282d9c4b: Pulling fs layer
9b0d3db6dc03: Pulling fs layer
8269c605f3f1: Pulling fs layer
6504d449e70c: Pulling fs layer
4e38f320d0d4: Pulling fs layer
b0a763e8ee03: Pulling fs layer
11917a028ca4: Pulling fs 

469/469 [==============================] - 2s 4ms/step - loss: 0.4239 - accuracy: 0.8693
Epoch 2/10

469/469 [==============================] - 2s 4ms/step - loss: 0.1928 - accuracy: 0.9434
Epoch 3/10

469/469 [==============================] - 2s 4ms/step - loss: 0.1511 - accuracy: 0.9552
Epoch 4/10

469/469 [==============================] - 2s 4ms/step - loss: 0.1279 - accuracy: 0.9620
Epoch 5/10

469/469 [==============================] - 2s 4ms/step - loss: 0.1117 - accuracy: 0.9664
Epoch 6/10

469/469 [==============================] - 2s 4ms/step - loss: 0.1024 - accuracy: 0.9685
Epoch 7/10

469/469 [==============================] - 2s 4ms/step - loss: 0.0933 - accuracy: 0.9711
Epoch 8/10

469/469 [==============================] - 2s 4ms/step - loss: 0.0888 - accuracy: 0.9718
Epoch 9/10

469/469 [==============================] - 2s 4ms/step - loss: 0.0835 - accuracy: 0.9732
Epoch 10/10

137/469 [=======>......................] - ETA: 1s - loss: 0.0655 - accuracy: 0.9796



Streaming azureml-logs/75_job_post-tvmps_f123fe9481f13ff58d15731b92e96095afc81d3d00e719a753089e9619330ad8_p.txt

bash: /azureml-envs/azureml_82b9ba677230e2f12409ea1ef3e08f3d/lib/libtinfo.so.5: no version information available (required by bash)
Entering job release. Current time:2020-07-08T09:55:35.586704
Starting job release. Current time:2020-07-08T09:55:36.615933
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 186
[2020-07-08T09:55:36.638922] Entering context manager injector.
Job release is complete. Current time:2020-07-08T09:55:38.516016

Execution Summary
RunId: keras-mnist_1594201764_d82072c2
Web View: https://ml.azure.com/experiments/keras-mnist/runs/keras-mnist_1594201764_d82072c2?wsid=/subscriptions/23b3eeaf-8619-495c-b837-5dece9a29613/resourcegroups/aml-rg1/workspaces/aml-ws



{'runId': 'keras-mnist_1594201764_d82072c2',
 'target': 'aml-compute',
 'status': 'Completed',
 'startTimeUtc': '2020-07-08T09:53:45.598745Z',
 'endTimeUtc': '2020-07-08T09:55:50.205998Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '2b8fc53a-ba95-4b87-ae97-f03d76bb1331',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'aml-compute',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'environment': {'name': 'Experiment keras-mnist Environment',
   'version': 'Autosave_2020-07-08T09:25:36Z_614620b5',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'condaDependenci

In [27]:
run.get_all_logs()

['/home/nbuser/library/usingTensorflowEstimator/training_folder/assets/keras-mnist_1594201764_d82072c2/azureml-logs/55_azureml-execution-tvmps_f123fe9481f13ff58d15731b92e96095afc81d3d00e719a753089e9619330ad8_p.txt',
 '/home/nbuser/library/usingTensorflowEstimator/training_folder/assets/keras-mnist_1594201764_d82072c2/azureml-logs/65_job_prep-tvmps_f123fe9481f13ff58d15731b92e96095afc81d3d00e719a753089e9619330ad8_p.txt',
 '/home/nbuser/library/usingTensorflowEstimator/training_folder/assets/keras-mnist_1594201764_d82072c2/azureml-logs/70_driver_log.txt',
 '/home/nbuser/library/usingTensorflowEstimator/training_folder/assets/keras-mnist_1594201764_d82072c2/azureml-logs/75_job_post-tvmps_f123fe9481f13ff58d15731b92e96095afc81d3d00e719a753089e9619330ad8_p.txt',
 '/home/nbuser/library/usingTensorflowEstimator/training_folder/assets/keras-mnist_1594201764_d82072c2/azureml-logs/process_info.json',
 '/home/nbuser/library/usingTensorflowEstimator/training_folder/assets/keras-mnist_1594201764_d820